In [0]:
%sh pip install kaggle
%sh
export KAGGLE_USERNAME=kn0wh3re
export KAGGLE_KEY=[YOUR_KAGGLE_KEY]
kaggle datasets download -d yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018

  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73053 sha256=ae599ee05e66482ed81ad4c4ff8aa7ae893b45ec2262b1eec0903bd08a075311
  Stored in directory: /root/.cache/pip/wheels/29/da/11/144cc25aebdaeb4931b231e25fd34b394e6a5725cbb2f50106
Successfully built kaggle
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
/bin/bash: line 1: fg: no job control

  0%|          | 0.00/1.95G [00:00<?, ?B/s]
  0%|          | 5.00M/1.95G [00:00<01:07, 30.9MB/s]
  0%|          | 9.00M/1.95G [00:00<01:08, 30.5MB/s]
  1%|          | 17.0M/1.95G [00:00<00:52, 39.1MB/s]
  1%|▏         | 25.0M/1.95G [00:00<00:44, 46.3MB/s]
  2%|▏         | 33.0M/1.95G [00:01<01:22, 25.0MB/s]
  2%|▏         | 41.0M/1.95G [00:01<01:05, 31.1MB/s]
  2%|▏         | 49.0M/1.95G [00:01<00:56, 36.1MB/s]
  3%|▎         | 57.0M/1.95G [00:01<00:51, 39.3MB/s]
  3%|▎         | 65.0M/1.95G [00:01<00:46, 43.8MB/s]
  4%|▎         | 73.0M/1.95G [00:01<00:40, 49.3

In [0]:
!mkdir airline_dataset
!unzip airline-delay-and-cancellation-data-2009-2018.zip -d airline_dataset

mkdir: cannot create directory ‘airline_dataset’: File exists
Archive:  airline-delay-and-cancellation-data-2009-2018.zip
  inflating: airline_dataset/2009.csv  
  inflating: airline_dataset/2010.csv  
  inflating: airline_dataset/2011.csv  
  inflating: airline_dataset/2012.csv  
  inflating: airline_dataset/2013.csv  
  inflating: airline_dataset/2014.csv  
  inflating: airline_dataset/2015.csv  
  inflating: airline_dataset/2016.csv  
  inflating: airline_dataset/2017.csv  
  inflating: airline_dataset/2018.csv  


In [0]:
%ls

airline-delay-and-cancellation-data-2009-2018.zip  ganglia/
airline_dataset/                                   logs/
conf/                                              metastore_db/
eventlogs/                                         preload_class.lst*


In [0]:
dbutils.fs.ls("file:/databricks/driver/airline_dataset/")

Out[22]: [FileInfo(path='file:/databricks/driver/airline_dataset/2017.csv', name='2017.csv', size=702257855, modificationTime=1565566180000),
 FileInfo(path='file:/databricks/driver/airline_dataset/2013.csv', name='2013.csv', size=787473333, modificationTime=1565565884000),
 FileInfo(path='file:/databricks/driver/airline_dataset/2012.csv', name='2012.csv', size=775286574, modificationTime=1565565804000),
 FileInfo(path='file:/databricks/driver/airline_dataset/2018.csv', name='2018.csv', size=892988892, modificationTime=1565566250000),
 FileInfo(path='file:/databricks/driver/airline_dataset/2015.csv', name='2015.csv', size=719068948, modificationTime=1565566038000),
 FileInfo(path='file:/databricks/driver/airline_dataset/2009.csv', name='2009.csv', size=792609318, modificationTime=1565565554000),
 FileInfo(path='file:/databricks/driver/airline_dataset/2014.csv', name='2014.csv', size=719599833, modificationTime=1565565964000),
 FileInfo(path='file:/databricks/driver/airline_dataset/2016

In [0]:
dbutils.fs.mkdirs("dbfs:/FileStore/data/airline_dataset")

dbutils.fs.cp("file:/databricks/driver/airline_dataset/", "dbfs:/FileStore/data/airline_dataset", recurse = True)

Out[23]: True

## Data Download checkpoint

In [0]:
from functools import reduce
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, ArrayType  

dataset_path = 'dbfs:/FileStore/data/airline_dataset/'

file_count = len(dbutils.fs.ls(dataset_path))
assert file_count == 10, f"Data not found, or filecount was not correct ({file_count}). You may want to check the path or run the notebook from start again"
file_count

Out[1]: 10

In [0]:
files = [fileinfo.path for fileinfo in dbutils.fs.ls(dataset_path)]
files

Out[2]: ['dbfs:/FileStore/data/airline_dataset/2009.csv',
 'dbfs:/FileStore/data/airline_dataset/2010.csv',
 'dbfs:/FileStore/data/airline_dataset/2011.csv',
 'dbfs:/FileStore/data/airline_dataset/2012.csv',
 'dbfs:/FileStore/data/airline_dataset/2013.csv',
 'dbfs:/FileStore/data/airline_dataset/2014.csv',
 'dbfs:/FileStore/data/airline_dataset/2015.csv',
 'dbfs:/FileStore/data/airline_dataset/2016.csv',
 'dbfs:/FileStore/data/airline_dataset/2017.csv',
 'dbfs:/FileStore/data/airline_dataset/2018.csv']

In [0]:
# # Build map of spark dataframes by reading json partition chunk files
# dataframes_map = map(lambda r: spark.read.csv(r, header=True), [f"{file}" for file in files])
# # reduce the dataframes into single dataframe by performing union over the mapped frames.
# df = reduce(lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True), dataframes_map)
df = spark.read.csv(files[0], header=True)
df.printSchema()

root
 |-- FL_DATE: string (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: string (nullable = true)
 |-- DEP_TIME: string (nullable = true)
 |-- DEP_DELAY: string (nullable = true)
 |-- TAXI_OUT: string (nullable = true)
 |-- WHEELS_OFF: string (nullable = true)
 |-- WHEELS_ON: string (nullable = true)
 |-- TAXI_IN: string (nullable = true)
 |-- CRS_ARR_TIME: string (nullable = true)
 |-- ARR_TIME: string (nullable = true)
 |-- ARR_DELAY: string (nullable = true)
 |-- CANCELLED: string (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: string (nullable = true)
 |-- CRS_ELAPSED_TIME: string (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: string (nullable = true)
 |-- AIR_TIME: string (nullable = true)
 |-- DISTANCE: string (nullable = true)
 |-- CARRIER_DELAY: string (nullable = true)
 |-- WEATHER_DELAY: strin

In [0]:
# df = spark.read.csv(files[0], header=True)
df = df.drop('Unnamed: 27')
df.show()

+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|
+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+
|2009-01-01|        XE|             1

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from pyspark.mllib.linalg import Vectors
from pyspark.ml.param import Param, Params
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer
from pyspark.ml import Pipeline

In [0]:
df = df.dropna(how="all")
df.show(2)

+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|
+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+
|2009-01-01|        XE|             1

In [0]:
plt.figure(figsize=(25, 12)).subplots_adjust(hspace = 0.5)

plt.subplot(2, 2 ,1)
df.groupby('FL_DATE_month').ARR_DELAY.sum().plot.bar().set_title('ARRIVAL delays by month')
plt.title('ARRIVAL delays by month', fontsize=16)
plt.ylabel('Hours', fontsize=14)
plt.xlabel('Month of the year', fontsize=14)

plt.subplot(2, 2 ,2)
df.groupby('FL_DATE_month').DEP_DELAY.sum().plot.bar()
plt.title('DEPARTURE delays by month', fontsize=16)
plt.ylabel('Hours', fontsize=14)
plt.xlabel('Month of the year', fontsize=14)

plt.show()

6429338


In [0]:
import pyspark.sql.functions as F

# df = df.withColumn("FL_DATE", F.col("FL_DATE").cast("timestamp"))

string_columns = ['FL_DATE', 'OP_CARRIER', 'ORIGIN', 'DEST', 'CANCELLATION_CODE']
numeric_columns = ['DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'WHEELS_ON', 'TAXI_IN', 'ARR_TIME', 'ARR_DELAY', 'DIVERTED', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'OP_CARRIER_FL_NUM', 'CRS_DEP_TIME', 'CRS_ARR_TIME']

for num_column in numeric_columns:
    df = df.withColumn(num_column, F.col(num_column).cast('float'))
df.printSchema()

root
 |-- FL_DATE: string (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: float (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: float (nullable = true)
 |-- DEP_TIME: float (nullable = true)
 |-- DEP_DELAY: float (nullable = true)
 |-- TAXI_OUT: float (nullable = true)
 |-- WHEELS_OFF: float (nullable = true)
 |-- WHEELS_ON: float (nullable = true)
 |-- TAXI_IN: float (nullable = true)
 |-- CRS_ARR_TIME: float (nullable = true)
 |-- ARR_TIME: float (nullable = true)
 |-- ARR_DELAY: float (nullable = true)
 |-- CANCELLED: string (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: float (nullable = true)
 |-- CRS_ELAPSED_TIME: float (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: float (nullable = true)
 |-- AIR_TIME: float (nullable = true)
 |-- DISTANCE: float (nullable = true)
 |-- CARRIER_DELAY: float (nullable = true)
 |-- WEATHER_DELAY: float (nullable = true

In [0]:
stages = [] # stages in our Pipeline

for categoricalCol in string_columns:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]


# Convert label into label indices using the StringIndexer
label_stringIdx =  StringIndexer(inputCol="CANCELLED", outputCol="label")
stages += [label_stringIdx]

assemblerInputs = [c + "classVec" for c in string_columns] + numeric_columns
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol='features')
stages += [assembler]

In [0]:
# Create a Pipeline.
pipeline = Pipeline().setStages(stages)
pipelineModel = pipeline.fit(df)
PreparedData = pipelineModel.transform(df)

In [0]:
PreparedData.show(1, truncate=False, vertical=True)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-2106842820896905> in <module>
----> 1 PreparedData.show(1, truncate=False, vertical=True)

NameError: name 'PreparedData' is not defined

In [0]:
stages

Out[11]: [StringIndexer_02011950a66f,
 OneHotEncoder_7cd298af794c,
 StringIndexer_35b627a5dde0,
 OneHotEncoder_194869b26a23,
 StringIndexer_fd1b170556ba,
 OneHotEncoder_8aa2c93ce26a,
 StringIndexer_c6a048543357,
 OneHotEncoder_74a189c0dba6,
 StringIndexer_173f8ec1ffbb,
 OneHotEncoder_68df842b7bbe,
 StringIndexer_e44bd0611674,
 VectorAssembler_c760fdd2cfce]